In [0]:
# gold_aggregate_vc.py
from pyspark.sql.functions import count, sum as sum_, col, regexp_replace

# Load the silver table as a DataFrame
silver_investments_vc = spark.table("workspace_elena_vc.default.silver_investments_vc")

# Clean and cast funding_total_usd to double
silver_investments_vc_clean = silver_investments_vc.withColumn(
    "funding_total_usd_clean",
    regexp_replace(col("funding_total_usd"), "[^0-9.]", "").cast("double")
)


"""
create Gold DataFrame with agg metrics:
    for each market, category_list, status
    - amount of invests
    - count of companies
"""
gold_investments_vc = (
        silver_investments_vc_clean.groupBy("market", "category_list", "status")
                 .agg(
                     count("name").alias("total_companies"),
                     sum_("funding_total_usd_clean").alias("total_amount_usd")
                 )
                 .orderBy(col("total_amount_usd").desc())
    )

gold_investments_vc.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace_elena_vc.default.gold_investments_vc")

